In [4]:
import numpy as np
import os 
import arviz as az
import pandas as pd
import math
import xarray as xr
os.chdir('..')
import waic as wc
import model_data as md

In [5]:
os.chdir('../data')
fnames = ['sockeye_data.csv']
df = md.load_data(fnames, reset_sessions = True)

C:\Users\brett\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [6]:
os.chdir('../Pystan/cmdstan_output')

In [7]:
cued_basic_fit = az.from_netcdf('cue_basic_fit.nc')
classic_basic_fit = az.from_netcdf('Classic_basic_classic_fit.nc')

cued_pscale_fit = az.from_netcdf('cue_pscale_fit.nc')
classic_pscale_fit = az.from_netcdf('Classic_pscale_fit.nc')

cued_pindep_fit = az.from_netcdf('cue_pindep_fit.nc')
classic_pindep_fit = az.from_netcdf('Classic_pindep_fit.nc')

In [8]:
task_list = df.groupby(['MSN'])['Subject'].unique()

In [9]:
task_list

MSN
RGT_ClassicA     [325, 326, 327, 328, 609, 610, 611, 612, 709, ...
RGT_ClassicB     [329, 330, 331, 332, 613, 614, 615, 616, 713, ...
RIGT_ClassicA                       [1001, 1002, 1016, 1011, 1012]
RIGT_ClassicB                                   [1015, 1009, 1010]
rGT_A-cue        [173, 174, 185, 186, 177, 178, 179, 180, 429, ...
rGT_B-cue        [175, 187, 188, 181, 182, 183, 184, 176, 425, ...
rGT_ClassicA     [301, 302, 317, 318, 319, 320, 333, 334, 335, ...
rGT_ClassicB     [307, 308, 321, 322, 323, 324, 337, 338, 339, ...
Name: Subject, dtype: object

In [10]:
classic_subs = np.concatenate(task_list[[task for task in df.MSN.unique() if 'Classic' in task]])
cued_subs = np.concatenate((task_list['rGT_A-cue'], task_list['rGT_B-cue']))

In [19]:
cued_subs

array([ 173,  174,  185,  186,  177,  178,  179,  180,  429,  430,  431,
        432,  417,  418,  419,  420,  501,  503,  507,  505,  524,  520,
        518,  522,  530,  532,  509,  513,  515,  511,  625,  626,  627,
        628,  617,  618,  619,  620, 1021, 1022, 1003, 1004, 1006, 1008,
       1101, 1102, 1103, 1104, 1205, 1206, 1229, 1230, 1221, 1222, 1311,
       1312, 1314, 1315, 1301, 1302, 1305, 1306, 1409, 1410, 1411, 1417,
       1428, 1401, 1402, 1403, 1509, 1511, 1513, 1515, 1501, 1503, 1505,
       1507,  175,  187,  188,  181,  182,  183,  184,  176,  425,  426,
        427,  428,  421,  422,  423,  424,  502,  506,  504,  508,  517,
        519,  523,  521,  531,  529,  510,  512,  514,  516,  629,  630,
        631,  632,  621,  622,  623,  624, 1028, 1023, 1024, 1005, 1007,
       1013, 1014, 1105, 1106, 1107, 1108, 1207, 1208, 1231, 1232, 1223,
       1224, 1309, 1310, 1313, 1316, 1303, 1304, 1307, 1308, 1413, 1414,
       1415, 1416, 1421, 1423, 1431, 1432, 1405, 14

In [16]:
from timeit import default_timer as timer

start = timer()

cued_basic_waic = wc.waic_fit(df, 5, cued_subs, cued_basic_fit, "basic")
end = timer()
print(end - start) # Time in seconds

loss_basic_waic

C:\Users\brett\sparklyRGT\Pystan\model_data.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Option'][i] = 0
C:\Users\brett\sparklyRGT\Pystan\model_data.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sub'][i] = sub_dict[df['Subject'][i]]


IndexError: index 3200 is out of bounds for axis 0 with size 3200

In [ ]:
cued_basic_waic = wc.waic_fit(df, 5, cued_subs, cued_basic_fit, "basic")
classic_basic_waic = wc.waic_fit(df, 5, classic_subs, classic_basic_fit, "basic")

basic_list = [loss_basic_waic.waic, outcome_basic_waic.waic, random_basic_waic.waic, reverse_basic_waic.waic, standard_basic_waic.waic, uncued_basic_waic.waic]
basic_list_SE = [loss_basic_waic.waic_se, outcome_basic_waic.waic_se, random_basic_waic.waic_se, reverse_basic_waic.waic_se, standard_basic_waic.waic_se, uncued_basic_waic.waic_se]

In [ ]:
loss_basicstar_waic = wc.waic_fit(df, 5, loss_subs, loss_basicstar_fit, "basicstar")
outcome_basicstar_waic = wc.waic_fit(df, 5, outcome_subs, outcome_basicstar_fit, "basicstar")
random_basicstar_waic = wc.waic_fit(df, 5, random_subs, random_basicstar_fit, "basicstar")
reverse_basicstar_waic = wc.waic_fit(df, 5, reverse_subs, reverse_basicstar_fit, "basicstar")
standard_basicstar_waic = wc.waic_fit(df, 5, standard_subs, standard_basicstar_fit, "basicstar")
uncued_basicstar_waic = wc.waic_fit(df, 5, uncued_subs, uncued_basicstar_fit, "basicstar")

basicstar_list = [loss_basicstar_waic.waic, outcome_basicstar_waic.waic, random_basicstar_waic.waic, reverse_basicstar_waic.waic, standard_basicstar_waic.waic, uncued_basicstar_waic.waic]
basicstar_list_SE = [loss_basicstar_waic.waic_se, outcome_basicstar_waic.waic_se, random_basicstar_waic.waic_se, reverse_basicstar_waic.waic_se, standard_basicstar_waic.waic_se, uncued_basicstar_waic.waic_se]

In [ ]:
loss_pscale_waic = wc.waic_fit(df, 5, loss_subs, loss_pscale_fit, "pscale")
outcome_pscale_waic = wc.waic_fit(df, 5, outcome_subs, outcome_pscale_fit, "pscale")
random_pscale_waic = wc.waic_fit(df, 5, random_subs, random_pscale_fit, "pscale")
reverse_pscale_waic = wc.waic_fit(df, 5, reverse_subs, reverse_pscale_fit, "pscale")
standard_pscale_waic = wc.waic_fit(df, 5, standard_subs, standard_pscale_fit, "pscale")
uncued_pscale_waic = wc.waic_fit(df, 5, uncued_subs, uncued_pscale_fit, "pscale")

pscale_list = [loss_pscale_waic.waic, outcome_pscale_waic.waic, random_pscale_waic.waic, reverse_pscale_waic.waic, standard_pscale_waic.waic, uncued_pscale_waic.waic]
pscale_list_SE = [loss_pscale_waic.waic_se, outcome_pscale_waic.waic_se, random_pscale_waic.waic_se, reverse_pscale_waic.waic_se, standard_pscale_waic.waic_se, uncued_pscale_waic.waic_se]

In [ ]:
loss_pscalestar_waic = wc.waic_fit(df, 5, loss_subs, loss_pscalestar_fit, "pscalestar")
outcome_pscalestar_waic = wc.waic_fit(df, 5, outcome_subs, outcome_pscalestar_fit, "pscalestar")
random_pscalestar_waic = wc.waic_fit(df, 5, random_subs, random_pscalestar_fit, "pscalestar")
reverse_pscalestar_waic = wc.waic_fit(df, 5, reverse_subs, reverse_pscalestar_fit, "pscalestar")
standard_pscalestar_waic = wc.waic_fit(df, 5, standard_subs, standard_pscalestar_fit, "pscalestar")
uncued_pscalestar_waic = wc.waic_fit(df, 5, uncued_subs, uncued_pscalestar_fit, "pscalestar")

pscalestar_list = [loss_pscalestar_waic.waic, outcome_pscalestar_waic.waic, random_pscalestar_waic.waic, reverse_pscalestar_waic.waic, standard_pscalestar_waic.waic, uncued_pscalestar_waic.waic]
pscalestar_list_SE = [loss_pscalestar_waic.waic_se, outcome_pscalestar_waic.waic_se, random_pscalestar_waic.waic_se, reverse_pscalestar_waic.waic_se, standard_pscalestar_waic.waic_se, uncued_pscalestar_waic.waic_se]

In [ ]:
loss_pindep_waic = wc.waic_fit(df, 5, loss_subs, loss_pindep_fit, "pindep")
outcome_pindep_waic = wc.waic_fit(df, 5, outcome_subs, outcome_pindep_fit, "pindep")
random_pindep_waic = wc.waic_fit(df, 5, random_subs, random_pindep_fit, "pindep")
reverse_pindep_waic = wc.waic_fit(df, 5, reverse_subs, reverse_pindep_fit, "pindep")
standard_pindep_waic = wc.waic_fit(df, 5, standard_subs, standard_pindep_fit, "pindep")
uncued_pindep_waic = wc.waic_fit(df, 5, uncued_subs, uncued_pindep_fit, "pindep")

pindep_list = [loss_pindep_waic.waic, outcome_pindep_waic.waic, random_pindep_waic.waic, reverse_pindep_waic.waic, standard_pindep_waic.waic, uncued_pindep_waic.waic]
pindep_list_SE = [loss_pindep_waic.waic_se, outcome_pindep_waic.waic_se, random_pindep_waic.waic_se, reverse_pindep_waic.waic_se, standard_pindep_waic.waic_se, uncued_pindep_waic.waic_se]

In [ ]:
loss_pindepstar_waic = wc.waic_fit(df, 5, loss_subs, loss_pindepstar_fit, "pindepstar")
outcome_pindepstar_waic = wc.waic_fit(df, 5, outcome_subs, outcome_pindepstar_fit, "pindepstar")
random_pindepstar_waic = wc.waic_fit(df, 5, random_subs, random_pindepstar_fit, "pindepstar")
reverse_pindepstar_waic = wc.waic_fit(df, 5, reverse_subs, reverse_pindepstar_fit, "pindepstar")
standard_pindepstar_waic = wc.waic_fit(df, 5, standard_subs, standard_pindepstar_fit, "pindepstar")
uncued_pindepstar_waic = wc.waic_fit(df, 5, uncued_subs, uncued_pindepstar_fit, "pindepstar")

pindepstar_list = [loss_pindepstar_waic.waic, outcome_pindepstar_waic.waic, random_pindepstar_waic.waic, reverse_pindepstar_waic.waic, standard_pindepstar_waic.waic, uncued_pindepstar_waic.waic]
pindepstar_list_SE = [loss_pindepstar_waic.waic_se, outcome_pindepstar_waic.waic_se, random_pindepstar_waic.waic_se, reverse_pindepstar_waic.waic_se, standard_pindepstar_waic.waic_se, uncued_pindepstar_waic.waic_se]

In [ ]:
waic_values = pd.DataFrame(
    {'basic_waic': basic_list,
     'basic_waic_SE': basic_list_SE, 
     'basicstar_waic': basicstar_list,
     'basicstar_waic_SE': basicstar_list_SE,
     'pscale_waic': pscale_list,
     'pscale_waic_SE': pscale_list_SE,
     'pscalestar_waic': pscalestar_list,
     'pscalestar_SE': pscalestar_list_SE,
     'pindep_waic': pindep_list,
     'pindep_waic_SE': pindep_list_SE,
     'pindepstar_waic': pindepstar_list,
     'pindepstar_waic_SE': pindepstar_list_SE},
     index = ["loss", "outcome", "random", "reverse", "standard", "uncued"])

waic_values.to_excel("waic_values.xlsx")